# Cortx Pytorch Integration

This project builds and demonstrates a pytorch integration. The project uses the Cortx framework to store some data. A Pytorch dataloader is created to download the data and use it for a classification task.

We use the Medical MNIST dataset from Kaggle available at https://www.kaggle.com/andrewmvd/medical-mnist

In [1]:
import os
import sys
import boto3
import logging
import shutil
import threading
from botocore.client import Config
from matplotlib import pyplot as plt
from botocore.exceptions import ClientError
from boto3.s3.transfer import TransferConfig

In [2]:
class ProgressPercentage(object):
    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()
        
    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write("\r%s  %s / %s  (%.2f%%)" %
                             (self._filename, self._seen_so_far, 
                              self._size, percentage))
            sys.stdout.flush()

def multipart_upload_with_s3(bucket_name, file_path=None, object_name=None):
    # Multipart upload (see notes)
    config = TransferConfig(multipart_threshold=1024 * 25, max_concurrency=10,
                            multipart_chunksize=1024 * 25, use_threads=True)
    key_path = 'multipart_files/{}'.format(object_name)
    print(bucket_name,file_path,object_name,key_path)
    s3_client.upload_file(file_path, bucket_name, key_path,
                          ExtraArgs={'ACL': 'public-read', 
                                     'ContentType': 'text/pdf'},
                          Config=config, Callback=ProgressPercentage(file_path))
    
def multipart_download_with_s3(bucket_name, file_path=None, object_name=None):
    config = TransferConfig(multipart_threshold=1024 * 25, max_concurrency=10,
                            multipart_chunksize=1024 * 25, use_threads=True)
    temp_file = os.path.dirname(__file__)
    s3_resource.Object(bucket_name, 
                       object_name
                       ).download_file(file_path, Config=config,
                                       Callback=ProgressPercentage(temp_file))

"""Functions for buckets operation"""
def create_bucket_op(bucket_name, region):
    if region is None:
        s3_client.create_bucket(Bucket=bucket_name)
    else:
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name, 
                                CreateBucketConfiguration=location)

def list_bucket_op(bucket_name, region, operation):
    buckets = s3_client.list_buckets()
    if buckets['Buckets']:
        for bucket in buckets['Buckets']:
            print(bucket)
            return True
    else:
        logging.error('unknown bucket operation')
        return False
    
def bucket_operation(bucket_name, region=None, operation='list'):
    try:
        if operation == 'delete':
            s3_client.delete_bucket(Bucket=bucket_name)
        elif operation == 'create':
            create_bucket_op(bucket_name, region)
        elif operation == 'list':
            return list_bucket_op(bucket_name, region, operation)
        else:
            logging.error('unknown bucket operation')
            return False
    except ClientError as e:
        logging.error(e)
        return False
    return True

"""Functions for objects operation"""
def list_object_op(bucket_name):
     s3_objects = s3_client.list_objects_v2(Bucket=bucket_name)
     if s3_objects.get('Contents'):
         for obj in s3_objects['Contents']:
             print(obj)

def delete_object_op(bucket_name, object_name, operation):
    if not object_name:
        logging.error('object_name missing for {}'.format(operation))
        return False  
    s3_client.delete_object(Bucket=bucket_name, Key=object_name)
    return True

def upload_download_object_op(bucket_name, object_name, file_path, operation):
    if not file_path or not object_name:
        logging.error('file_path and/or object_name missing for upload')
        return False
    if operation == 'upload':
        multipart_upload_with_s3(bucket_name=bucket_name, file_path=file_path,
                                 object_name=object_name)
    else:
        multipart_download_with_s3(bucket_name=bucket_name, file_path=file_path,
                                   object_name=object_name)
    return True

def object_operation(bucket_name=None, object_name=None, file_path=None,
                     operation='list'):                                                             
    try:
        if not bucket_name:
            logging.error('The bucket name %s is missing for %s operation!'
                          % (bucket_name, operation))
            return False
        if operation == 'list':
            list_object_op(bucket_name)
        elif operation == 'delete':
            return delete_object_op(bucket_name, object_name, operation)
        elif operation == 'upload' or operation == 'download':
            return upload_download_object_op(bucket_name, object_name,
                                             file_path, operation)      
        else:
            logging.error('unknown object operation')
            return False
    except ClientError as e:
        logging.error(e)
        return False
    return True

"""Functions for files operation"""
def list_op_file(bucket_name):
    current_bucket = s3_resource.Bucket(bucket_name)
    print('The files in bucket %s:\n' % (bucket_name))
    for obj in current_bucket.objects.all():
        print(obj.meta.data) 
        
    return True

def delete_op_file(bucket_name, file_name, operation):
    if not file_name:
        logging.error('The file name %s is missing for%s operation!' 
                      % (file_name, operation))
        return False
    s3_client.delete_object(Bucket=bucket_name, Key=file_name)
    return True
    
def upload_download_op_file(bucket_name, file_name, file_location,
                            region, operation):
    if not file_location:
        logging.error('The file location %d is missing for %s operation!'
                      % (file_location, operation))
        return False
    if operation == 'download':
        s3_resource.Bucket(bucket_name).download_file(file_name, file_location)
    elif operation == 'upload' and region is None:
        s3_resource.Bucket(bucket_name).upload_file(file_location, file_name)
    else:
         location = {'LocationConstraint': region}
         s3_resource.Bucket(bucket_name
                            ).upload_file(file_location, file_name,
                                          CreateBucketConfiguration=location) 
    return True
    
def file_operation(bucket_name=None, file_name=None, file_location=None, 
                   region=None, operation='list'):
    if not bucket_name:
        logging.error('The bucket name is %s missing!' % (bucket_name))
        return False 
    try:
        if operation == 'list':
            return list_op_file(bucket_name)
        elif operation == 'delete':
            return delete_op_file(bucket_name, file_name, operation)  
        elif operation == 'upload' or operation == 'download':
            return upload_download_op_file(bucket_name, file_name, 
                                           file_location, region, operation)
        else:
            logging.error('unknown file operation')
            return False  
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [3]:
END_POINT_URL = 'http://uvo17qqh4jn92xmchpj.vm.cld.sr'
A_KEY = 'AKIAtEpiGWUcQIelPRlD1Pi6xQ'
S_KEY = 'YNV6xS8lXnCTGSy1x2vGkmGnmdJbZSapNXaSaRhK'

In [4]:
s3_resource = boto3.resource('s3', endpoint_url=END_POINT_URL,
                             aws_access_key_id=A_KEY,
                             aws_secret_access_key=S_KEY,
                             config=Config(signature_version='s3v4'),
                             region_name='US')

s3_client = boto3.client('s3', endpoint_url=END_POINT_URL,
                         aws_access_key_id=A_KEY,
                         aws_secret_access_key=S_KEY,
                         config=Config(signature_version='s3v4'),
                         region_name='US')

In [40]:
list_op_file('headct')

The files in bucket headct:

{'Key': '003332.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 34, tzinfo=tzutc()), 'ETag': '"520d8ea45f112a33f5f97d39b8df9e79"', 'Size': 1045, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '003424.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 32, tzinfo=tzutc()), 'ETag': '"b41954570a6f85f5e42affa9740791a2"', 'Size': 1344, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '003904.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 37, tzinfo=tzutc()), 'ETag': '"71122c5c904fb9c58a780c9eb61ede41"', 'Size': 1219, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '004277.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 35

True

## Create MEDMINST data buckets on Cortx

In [9]:
class_names = ['abdomenct', 'breastmri', 'chestct', 'cxr', 'hand', 'headct']

In [33]:
for class_name in class_names:
    bucket_operation(class_name, None, 'create')
    print('created bucket for class: {}'.format(class_name))


ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: The bucket you tried to create already exists, and you own it.


created bucket for class: abdomenct


ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: The bucket you tried to create already exists, and you own it.


created bucket for class: breastmri


ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: The bucket you tried to create already exists, and you own it.


...

created bucket for class: cxr


ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: The bucket you tried to create already exists, and you own it.


created bucket for class: hand


ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: The bucket you tried to create already exists, and you own it.


created bucket for class: headct


In [14]:
bucket_operation(None, None, operation='list')

{'Name': 'abdomenct', 'CreationDate': datetime.datetime(2021, 4, 27, 23, 2, 19, tzinfo=tzutc())}


True

In [26]:
dataset_base_path = '/home/dreamchild/sgcortex/data/medmnist/'

In [27]:
local_class_dirs = os.listdir(dataset_base_path)

In [28]:
local_class_dirs

['Hand', 'CXR', 'AbdomenCT', 'BreastMRI', 'HeadCT', 'ChestCT']

In [34]:
local_class_dir_files = {}

for class_dir in local_class_dirs:
    class_dir_files = []
    for file in os.listdir(dataset_base_path + class_dir)[:10]:
        class_dir_files.append(file)
    local_class_dir_files[class_dir] = class_dir_files

In [35]:
local_class_dir_files

{'Hand': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '009181.jpeg',
  '009386.jpeg',
  '003332.jpeg',
  '004277.jpeg',
  '007914.jpeg',
  '003904.jpeg'],
 'CXR': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '009181.jpeg',
  '009386.jpeg',
  '003332.jpeg',
  '004277.jpeg',
  '007914.jpeg',
  '003904.jpeg'],
 'AbdomenCT': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '009181.jpeg',
  '009386.jpeg',
  '003332.jpeg',
  '004277.jpeg',
  '007914.jpeg',
  '003904.jpeg'],
 'BreastMRI': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '003332.jpeg',
  '004277.jpeg',
  '007914.jpeg',
  '003904.jpeg',
  '000689.jpeg',
  '005112.jpeg'],
 'HeadCT': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '009181.jpeg',
  '009386.jpeg',
  '003332.jpeg',
  '004277.jpeg',
  '007914.jpeg',
  '003904.jpeg'],
 'ChestCT': ['005383.jpeg',
  '008405.jpeg',
  '003424.jpeg',
  '006537.jpeg',
  '009181.jpeg'

### Upload (5) files from each category into cortx

In [38]:
for k in local_class_dir_files.keys():
    for file in local_class_dir_files[k]:
        upload_file_path = dataset_base_path + k + '/' + file
        file_operation(k.lower(), file, upload_file_path, None, 'upload')
        print('uploaded file {} to bucket {}'.format(upload_file_path, k.lower()))

uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/005383.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/008405.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/003424.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/006537.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/009181.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/009386.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/003332.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/004277.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/007914.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/Hand/003904.jpeg to bucket hand
uploaded file /home/dreamchild/sgcortex/data/medmnist/CXR/005383.jpeg to bucket cxr
uploaded file /home/dreamchild/sgcortex/data/medmnist/CX

### Setup Pytorch dataloader

In [42]:
file_operation('hand',operation='list')

The files in bucket hand:

{'Key': '003332.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 6, tzinfo=tzutc()), 'ETag': '"7aab34fe6035251e8bd656c28ce96ef6"', 'Size': 1357, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '003424.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 4, tzinfo=tzutc()), 'ETag': '"f9207043d0836eac5d7a282a4fffa152"', 'Size': 1555, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '003904.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 8, tzinfo=tzutc()), 'ETag': '"1e1331b574296a60aff80e0247ba9f7b"', 'Size': 1824, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': '004277.jpeg', 'LastModified': datetime.datetime(2021, 4, 27, 23, 50, 7, tzin

True

In [81]:
class CortxDataLoader:
    
    def __init__(self, endpoint = None, accessKey = None, serviceKey = None, class_buckets = []):
        self.endpoint = endpoint
        self.accessKey = accessKey
        self.serviceKey = serviceKey
        self.class_buckets = class_buckets
        
    def download_dataset(self, destination_dir = None):
        for bucket in self.class_buckets:
            s3_bucket = s3_resource.Bucket(bucket)
            
            #files_in_bucket = s3_resource.Bucket(bucket).objects.all()
            for fileObj in s3_bucket.objects.all():
                filename = fileObj.key
                if not os.exists('/home/dreamchild/sgcortex/medmnist_dl/' + bucket):
                    os.mkdir('/home/dreamchild/sgcortex/medmnist_dl/' + bucket)
                with open('/home/dreamchild/sgcortex/medmnist_dl/' + bucket + '/' + filename, 'wb') as f:
                    s3_bucket.download_fileobj(filename, f)
                    
                #s3_bucket.download()
                # file_operation(bucket, file., destination_dir + '/' + file['Key'], None, operation='download')
                print('found file: {}'.format(filename))
                
    
    def train_data_loader(self):
        pass
    
    def test_data_loader(self):
        pass
    
    def val_data_loader(self):
        pass

In [82]:
cortx_loader = CortxDataLoader(class_buckets=['hand'])

In [83]:
cortx_loader.download_dataset('/home/dreamchild/sgcortex/data/medmnist-dl')

ConnectTimeoutError: Connect timeout on endpoint URL: "http://uvo17qqh4jn92xmchpj.vm.cld.sr/hand?encoding-type=url"